# Customer Retention Rates: Part 2 of Store Analysis
#### by ***Wayne Omondi***

Any company that wants to succeed must keep a close eye on its customer retention metrics.

There’s a simple, economic reason why customer retention is so important: Keeping your existing customers is a lot less expensive than trying to win new ones. Loyal customers also contribute to your business’ health by providing referrals, promoting your brand on social media, and giving feedback to improve your product or service. So, it’s critical for companies to keep an eye on their customer retention rate.

How do you calculate customer retention rate?
To determine your retention rate, first identify the time frame you want to study
Next, collect the number of existing customers at the start of the time period (S)
Then find the number of total customers at the end of the time period (E)

one of the best things for any business is to establish returning customers
For this we will explore the concept of a ***cohort***

A cohort is a group of people that share a characteristics
A cohort has three main ascpects 
1. time
2. size
3. behaviour

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings("ignore")

We will use the SuperStore dataset from [Part 1. EDA Project](https://github.com/WayneNyariroh/StoreSales_Analysis)

In [5]:
store_df = pd.read_csv(r"C:\Users\FatherMammoth\Documents\PortfolioPorjects\SalesAnalysis\data\SuperStoreSales_Whole.csv")

In [6]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
 18  Quantity